In [ ]:
import sys
sys.path.append("python/")

import sympy as sp
import numdifftools as nd
import numpy as np
import scipy.optimize as opt

import datlib
import symbase
import Z2sym
import Z3sym
import Z3num
import Z2num

# Ферромагнетик в симметричном случае

## Выражение для $\cal  Z^{(3)}$

### Аргументы

In [ ]:
hi, hj, lij, lik = Z3sym.hi, Z3sym.hj, Z3sym.lij, Z3sym.lik 
s1, s2, s3 = Z3sym.s1, Z3sym.s2, Z3sym.s3
x, t = sp.symbols("x t")

In [ ]:
arg_exp = hj*x
arg_exp = arg_exp.replace(
    x, sp.solve(sp.sqrt(hi**2+lij**2+2*hi*lij*x)-t, x)[0]
).expand()
arg_exp += (s1+s2)*t + s1*s2*lik
expr_limit = hi+s3*lij
arg_F2 = (t+(s1+s2)*lik)/sp.sqrt(2*lik)

arg_exp = arg_exp.replace(t, t-(s1+s2)*lij*hi/hj).expand()
arg_F2 = arg_F2.replace(t, t-(s1+s2)*lij*hi/hj).expand()
expr_limit += (s1+s2)*lij*hi/hj

arg_exp = arg_exp.replace(t, t*sp.sqrt(2*lij*hi/hj)).expand()
arg_F2 = arg_F2.replace(t, t*sp.sqrt(2*lij*hi/hj)).expand()
expr_limit /= sp.sqrt(2*lij*hi/hj)

display(arg_exp)
display(arg_F2)
display(expr_limit)

In [ ]:
arg_exp -= t**2
arg_exp += expr_limit**2
arg_exp = arg_exp.expand()
display(arg_exp)

### Производные $F^{(3)}$

In [ ]:
t, x, a, b = sp.symbols("t x a b")

In [ ]:
Z2sym.F2(a*t+b).diff(t).expand()

In [ ]:
def get_I2_n(x, n):
    """I^{(2)}_n(x)"""
    _expr_n_0 = 1/sp.sqrt(a)*Z2sym.F2(sp.sqrt(a)*x)*sp.exp(x**2*(a-1))
    _expr_n_1 = 1/(2*a)*sp.exp(-x**2)*(sp.exp(x**2*a)-1)

    k = n // 2
    if n % 2 == 0:
        return _expr_n_0.diff(a, k).replace(a, 1).expand()
    else:
        return _expr_n_1.diff(a, k).replace(a, 1).expand()

In [ ]:
get_I2_n(x, 4)

In [ ]:
class F3_dummy(sp.Function):
    def _latex(self, printer, exp=None):
        arg1, arg2, arg3 = list(map(printer.doprint, self.args))
        return "F_{dummy}^{(3)}"+f"\\left({arg1} \, | \, {arg2}, {arg3} \\right)" 

In [ ]:
def get_I3_n(x, a, b, n):
    """I^{(3)}_n(x, a, b)"""
    if a == 1:
        if b == 0:
            result = x**(n+1)*Z2sym.F2(x)
            result -= get_I2_n(x, n+1)
            return result/(n+1)

        # b != 0
        result = -x**n*Z2sym.F2(x+b)
        result += get_I2_n(x, n)

        if n == 0:
            result += sp.exp(-x**2)*Z2sym.F2(b)
        if n != 0:
            result += n*get_I3_n(x, a, b, n-1)

        return result/(2*b)

    # a != 1, b != 0
    if n == 0:
        return F3_dummy(x, a, b)

    result = x**(n-1)*Z2sym.F2(a*x+b)
    result -= a*get_I2_n(x, n-1)
    result += 2*a*b*get_I3_n(x, a, b, n-1)

    if n == 1:
        result -= sp.exp(-x**2)*Z2sym.F2(b)
    if n != 1:
        result -= (n-1)*get_I3_n(x, a, b, n-2)
    
    return result/(2*(1-a**2))

In [ ]:
get_I3_n(x, a, b, 1).expand()

In [ ]:
class F3(sp.Function):
    def _latex(self, printer, exp=None):
        arg1, arg2, arg3 = list(map(printer.doprint, self.args))
        return "F^{(3)}"+f"\\left({arg1} \, | \, {arg2}, {arg3} \\right)" 

    def fdiff(self, argindex=1):
        if argindex == 1:
            x, a, b = self.args
            return Z2sym.F2(a*x+b)-2*x*F3(*self.args)
        
        t, x, a, b = sp.symbols("t x a b")
        expr = Z2sym.F2(a*t+b).diff([a, b][argindex-2]).expand()
        expr = sp.collect(expr, [Z2sym.F2(a*t+b)], evaluate=False)

        I3 = expr[Z2sym.F2(a*t+b)]
        I2 = expr[1]

        expr = 0
        I3 = sp.Poly(I3, t).as_dict()
        for key, item in I3.items():
            expr += item*get_I3_n(x, a, b, key[0])
        I2 = sp.Poly(I2, t).as_dict()
        for key, item in I2.items():
            expr += item*get_I2_n(x, key[0])

        subs = dict(zip([x, a, b], self.args))
        subs.update({F3_dummy(x, a, b): F3(*self.args)})
        return expr.subs(subs) 

In [ ]:
F3(x, a, b).diff(b)

In [ ]:
class F3_degenerate(sp.Function):
    def _latex(self, printer, exp=None):
        arg1, arg2, arg3, arg4, arg5 = list(map(printer.doprint, self.args))
        return "F_{deg,"+f"{arg4},{arg5}"+"}^{(3)}"+f"\\left({arg1} \, | \, {arg2}, {arg3} \\right)" 
    
    def fdiff(self, argindex=1):
        if argindex == 1:
            x, a, b = sp.symbols("x a b")
            subs = dict(zip([x, a, b], self.args[:3]))
            expr = Z2sym.F2(a*x+b).diff(a, self.args[3]).diff(b, self.args[4])

            x, a, b, _, _ = self.args
            return expr.subs(subs) -2*x*F3_degenerate(*self.args)
        
        args = list(self.args)
        args[argindex+1] += 1
        return F3_degenerate(*args)
    

def evaluate_F3_degenerate(expr):
    def evaluate(*args):
        t, x, a, b = sp.symbols("t x a b")
        expr = Z2sym.F2(a*t+b).diff(a, args[3])
        expr = expr.diff(b, args[4]).expand()
        expr = sp.collect(expr, [Z2sym.F2(a*t+b)], evaluate=False)
        
        I3 = expr.get(Z2sym.F2(a*t+b), sp.Integer(0)).subs({a: 1, b: 0})
        I2 = expr.get(1, sp.Integer(0)).subs({a: 1, b: 0})

        expr = 0
        I3 = sp.Poly(I3, t).as_dict()
        for key, item in I3.items():
            expr += item*get_I3_n(x, 1, 0, key[0])
        I2 = sp.Poly(I2, t).as_dict()
        for key, item in I2.items():
            expr += item*get_I2_n(x, key[0])

        subs = dict(zip([x, a, b], args[:3]))
        return expr.subs(subs) 
    
    return expr.replace(F3_degenerate, evaluate)

In [ ]:
expr = F3_degenerate(x, a, b, 0, 0).diff(x, a)
expr = evaluate_F3_degenerate(expr)

expr

### Производные $\cal F^{(3)}$

In [ ]:
arg_x = (hi+s3*lij+(s1+s2)*hi/hj*lij)/sp.sqrt(2*hi/hj*lij)
arg_a = sp.sqrt(hi/hj*lij)/sp.sqrt(lik)
arg_b = (s1+s2)/sp.sqrt(2*lik)*(lik-hi/hj*lij)
arg_expr = s3*hj+s1*s2*lik+(s1+s2)*(hi+s3*lij)

In [ ]:
expr = arg_x*arg_a+arg_b
expr.factor()

In [ ]:
expr = arg_expr-arg_x**2
expr = expr.expand()
expr

In [ ]:
_sigmas_expr_E = sp.exp(arg_expr).expand()
_sigmas_expr_F = Z2sym.F2(arg_x).expand()
_sigmas_expr_FTilde = Z2sym.F2(arg_x*arg_a+arg_b).expand()
_sigmas_expr_cF3 = F3(arg_x, arg_a, arg_b).expand()

_sigmas_terms = {
    _sigmas_expr_E*_sigmas_expr_cF3: sp.Dummy(),
    _sigmas_expr_E*_sigmas_expr_F: sp.Dummy(),
    _sigmas_expr_E*_sigmas_expr_FTilde: sp.Dummy(),
    _sigmas_expr_E: sp.Dummy()
}

In [ ]:
class Z3SigmasFunctionBase(sp.Function):
    def __init__(self, name, *args, **kwargs):
        self.name = name

    def _latex(self, printer, exp=None):
        args = list(map(printer.doprint, self.args[:4]))
        args = f"\\left({args[0]}, {args[1]}, {args[2]}, {args[3]}\\right)"

        indexes = ""
        for i, item in enumerate(self.args[4:]):
            if item:
                indexes += str(i+1)
        name = f"{self.name}"+"_{" + indexes + "}"

        if exp is None:
            return name+args
        else:
            exp = printer.doprint(exp)
            return f"{name}^{exp}"+args

    def get_sigmas_factor(self):
        factor = 1
        if self.args[4]:
            factor *= s1
        if self.args[5]:
            factor *= s2
        if self.args[6]:
            factor *= s3
        return factor

In [ ]:
class EFunction(Z3SigmasFunctionBase):
    def __init__(self, *args, **kwargs):
        super().__init__("{\\cal E}", *args, **kwargs)

    def fdiff(self, argindex=1):
        factor = self.get_sigmas_factor()
        expr = (factor*_sigmas_expr_E).diff([hi, hj, lij, lik][argindex-1])
        terms = symbase.collect_sigmas(expr, _sigmas_terms, [s1, s2, s3])

        subs = dict(zip([hi, hj, lij, lik], self.args[:4]))

        result = 0
        for key, item in terms[_sigmas_expr_E].items():
            result += item.subs(subs)*EFunction(*self.args[:4], *key)

        return result

    @classmethod
    def eval(cls, *args):
        if (not args[4]) and args[5]:
            return EFunction(*args[:4], 1, 0, args[-1])

    def _eval_rewrite(self, rule, args, **hints):
        args_subs = dict(zip([hi, hj, lij, lik], self.args[:-3]))
        if rule == sp.exp:
            _sigmas_expr = self.get_sigmas_factor()*_sigmas_expr_E
            result = 0
            for is1 in Z3sym.SIGMAS:
                for is2 in Z3sym.SIGMAS:
                    for is3 in Z3sym.SIGMAS:
                        is_subs = dict(zip([s1, s2, s3], [is1, is2, is3]))
                        result += _sigmas_expr.subs(args_subs).subs(is_subs)
            return result

In [ ]:
display(EFunction(hi, hj, lij, lik, 1, 1, 1).diff(hi).rewrite(sp.exp)-EFunction(hi, hj, lij, lik, 1, 1, 1).rewrite(sp.exp).diff(hi))

In [ ]:
class FFunction(Z3SigmasFunctionBase):
    def __init__(self, *args, **kwargs):
        super().__init__("{\\cal F}", *args, **kwargs)

    def fdiff(self, argindex=1):
        factor = self.get_sigmas_factor()
        expr = (factor*_sigmas_expr_E*_sigmas_expr_F).diff([hi, hj, lij, lik][argindex-1])
        terms = symbase.collect_sigmas(expr, _sigmas_terms, [s1, s2, s3])

        subs = dict(zip([hi, hj, lij, lik], self.args[:4]))

        result = 0
        if _sigmas_expr_E in terms:
            for key, item in terms[_sigmas_expr_E].items():
                result += item.subs(subs)*EFunction(*self.args[:4], *key)

        for key, item in terms[_sigmas_expr_E*_sigmas_expr_F].items():
            result += item.subs(subs)*FFunction(*self.args[:4], *key)

        return result

    @classmethod
    def eval(cls, *args):
        if (not args[4]) and args[5]:
            return FFunction(*args[:4], 1, 0, args[-1])

    def _eval_rewrite(self, rule, args, **hints):
        args_subs = dict(zip([hi, hj, lij, lik], self.args[:-3]))
        if rule == sp.exp:
            _sigmas_expr = self.get_sigmas_factor()*_sigmas_expr_E*_sigmas_expr_F
            result = 0
            for is1 in Z3sym.SIGMAS:
                for is2 in Z3sym.SIGMAS:
                    for is3 in Z3sym.SIGMAS:
                        is_subs = dict(zip([s1, s2, s3], [is1, is2, is3]))
                        result += _sigmas_expr.subs(args_subs).subs(is_subs)
            return result

In [ ]:
display(FFunction(hi, hj, lij, lik, 1, 1, 1).diff(lik).rewrite(sp.exp)-FFunction(hi, hj, lij, lik, 1, 1, 1).rewrite(sp.exp).diff(lik))

In [ ]:
class FTildeFunction(Z3SigmasFunctionBase):
    def __init__(self, *args, **kwargs):
        super().__init__( "\\tilde{\\cal F}", *args, **kwargs)

    def fdiff(self, argindex=1):
        factor = self.get_sigmas_factor()
        expr = (factor*_sigmas_expr_E*_sigmas_expr_FTilde).diff([hi, hj, lij, lik][argindex-1])
        terms = symbase.collect_sigmas(expr, _sigmas_terms, [s1, s2, s3])

        subs = dict(zip([hi, hj, lij, lik], self.args[:4]))

        result = 0
        if _sigmas_expr_E in terms:
            for key, item in terms[_sigmas_expr_E].items():
                result += item.subs(subs)*EFunction(*self.args[:4], *key)

        for key, item in terms[_sigmas_expr_E*_sigmas_expr_FTilde].items():
            result += item.subs(subs)*FTildeFunction(*self.args[:4], *key)

        return result

    @classmethod
    def eval(cls, *args):
        if (not args[4]) and args[5]:
            return FTildeFunction(*args[:4], 1, 0, args[-1])

    def _eval_rewrite(self, rule, args, **hints):
        args_subs = dict(zip([hi, hj, lij, lik], self.args[:-3]))
        if rule == sp.exp:
            _sigmas_expr = self.get_sigmas_factor()*_sigmas_expr_E*_sigmas_expr_FTilde
            result = 0
            for is1 in Z3sym.SIGMAS:
                for is2 in Z3sym.SIGMAS:
                    for is3 in Z3sym.SIGMAS:
                        is_subs = dict(zip([s1, s2, s3], [is1, is2, is3]))
                        result += _sigmas_expr.subs(args_subs).subs(is_subs)
            return result

In [ ]:
expr = FTildeFunction(hi, hj, lij, lik, 1, 1, 1).diff(hi).rewrite(sp.exp)
expr -= FTildeFunction(hi, hj, lij, lik, 1, 1, 1).rewrite(sp.exp).diff(hi)
expr = expr.factor()
expr

In [ ]:
class cF3(sp.Function):
    def _latex(self, printer, exp=None):
        args = list(map(printer.doprint, self.args[:4]))
        args = f"\\left({args[0]}, {args[1]}, {args[2]}, {args[3]}\\right)"
        if exp is None:
            return "{\\cal F}^{(3)}"+args
        else:
            exp = printer.doprint(exp)
            return f"\\left({self._latex(printer)}\\right)^{exp}"
    
    def fdiff(self, argindex=1):
        expr = (s1*s2*s3*_sigmas_expr_E*_sigmas_expr_cF3).diff([hi, hj, lij, lik][argindex-1])
        terms = symbase.collect_sigmas(expr, _sigmas_terms, [s1, s2, s3])
        
        subs = dict(zip([hi, hj, lij, lik], self.args[:4]))

        result = 0
        for key, item in terms[_sigmas_expr_E].items():
            result += item.factor().subs(subs)*EFunction(*self.args[:4], *key)

        for key, item in terms[_sigmas_expr_E*_sigmas_expr_F].items():
            result += item.factor().subs(subs)*FFunction(*self.args[:4], *key)
        
        for key, item in terms[_sigmas_expr_E*_sigmas_expr_FTilde].items():
            result += item.factor().subs(subs)*FTildeFunction(*self.args[:4], *key)
        
        for key, item in terms[_sigmas_expr_E*_sigmas_expr_cF3].items():
            if key != (1, 1, 1):
                raise RuntimeError
            result += item.factor().subs(subs)*cF3(*self.args)

        return result

    def _eval_rewrite(self, rule, args, **hints):
        args_subs = dict(zip([hi, hj, lij, lik], self.args[:-3]))
        if rule == sp.exp:
            _sigmas_expr = s1*s2*s3*_sigmas_expr_E*_sigmas_expr_cF3
            result = 0
            for is1 in Z3sym.SIGMAS:
                for is2 in Z3sym.SIGMAS:
                    for is3 in Z3sym.SIGMAS:
                        is_subs = dict(zip([s1, s2, s3], [is1, is2, is3]))
                        result += _sigmas_expr.subs(args_subs).subs(is_subs)
            return result

In [ ]:
sp.Add(*map(lambda x: x*4*lij*hi**2*sp.sqrt(hj)*(lij*hi-lik*hj), cF3(hi, hj, lij, lik).diff(hi).args))

In [ ]:
class cF3_degenerate(sp.Function):
    def _latex(self, printer, exp=None):
        args = list(map(printer.doprint, self.args))
        value_args = f"\\left({args[0]}, {args[1]}, {args[2]}, {args[3]}\\right)"
        diffs_args = f"{args[4]}, {args[5]}, {args[6]}, {args[7]}"
        if exp is None:
            return "{\\cal F}_{deg,"+diffs_args+"}"+value_args
        else:
            exp = printer.doprint(exp)
            return f"\\left({self._latex(printer)}\\right)^{exp}"
    
    def fdiff(self, argindex=1):    
        args = list(self.args)
        args[argindex+3] += 1
        return cF3_degenerate(*args)

def evaluate_cF3_degenerate(expr):
    def evaluate(*args):
        expr = (s1*s2*s3*_sigmas_expr_E*F3_degenerate(arg_x, arg_a, arg_b, 0, 0)).diff(hi, args[4])
        expr = expr.diff(hj, args[5]).diff(lij, args[6]).diff(lik, args[7])
        
        expr = evaluate_F3_degenerate(expr)

        terms = symbase.collect_sigmas(expr, _sigmas_terms, [s1, s2, s3])
        
        subs = dict(zip([hi, hj, lij, lik], args[:4]))
        
        result = 0
        for key, item in terms[_sigmas_expr_E].items():
            result += item.factor().subs(subs)*EFunction(*args[:4], *key)

        for key, item in terms[_sigmas_expr_E*_sigmas_expr_F].items():
            result += item.factor().subs(subs)*FFunction(*args[:4], *key)

        if _sigmas_expr_E*_sigmas_expr_FTilde in terms:
            for key, item in terms[_sigmas_expr_E*_sigmas_expr_FTilde].items():
                result += item.factor().subs(subs)*FFunction(*args[:4], *key)

        return result
    
    return expr.replace(cF3_degenerate, evaluate)

In [ ]:
# expr = cF3_degenerate(hi, hj, lij, lik, *(4*[0])).diff(lij, 1)
# expr = evaluate_cF3_degenerate(expr)

# expr

### Расчет $F^{(2)}$ и $F^{(3)}$

#### Определения

In [ ]:
class erfcx(sp.Function):
    def _latex(self, printer, exp=None):
        arg = printer.doprint(self.args[0])
        if exp is None:
            return "\operatorname{erfcx}"+f"\\left({arg}\\right)"
        else:
            exp = printer.doprint(exp)
            return f"\\left({self._latex(printer)}\\right)^{exp}"

In [ ]:
def erfi_to_dawsn(expr):
    def __helper(z):
        return Z2sym.F2(z)*2/sp.sqrt(sp.pi)*sp.exp(z**2)
    return expr.replace(sp.erfi, __helper)


def erfc_to_erfcx(expr):
    def __helper(z):
        return erfcx(z)*sp.exp(-z**2)
    return expr.replace(sp.erfc, __helper)

In [ ]:
t, x, a, b = sp.symbols("t x a b")
n, h = sp.symbols("n h")
n0 = sp.Symbol("n_0")

#### $c > 0$

In [ ]:
c = sp.Symbol("c", positive=True)
expr = t**2-(a*t+b-n*h)**2
expr = expr.expand()
expr = expr.replace(a**2, 1-c).expand()
expr = sp.integrate(sp.exp(expr), [t, 0, x])*sp.exp(-x**2)/(n*sp.sqrt(sp.pi))
expr = expr.expand()
expr = erfi_to_dawsn(expr)
expr = sp.Add(*map(lambda x: x.simplify(), expr.args))
expr

In [ ]:
print(sp.solve(expr.args[0].args[-1].diff(n), n)[0])

In [ ]:
print(sp.solve(expr.args[1].args[-1].diff(n), n)[0])

In [ ]:
expr = -2*a*b*x + 2*a*h*n*x - b**2 + 2*b*h*n + c*x**2 - h**2*n**2 - x**2
expr = expr.replace(n, n0+n).expand()
expr = expr.collect(n)
sp.print_latex(expr)
expr

In [ ]:
expr = -b**2 + 2*b*h*n - h**2*n**2 - x**2
expr = expr.replace(n, n0+n).expand()
expr = expr.collect(n)
sp.print_latex(expr)
expr

#### $c < 0$

In [ ]:
c = sp.Symbol("c", negative=True)
expr = t**2-(a*t+b-n*h)**2
expr = expr.expand()
expr = expr.replace(a**2, 1-c).expand()
expr = sp.integrate(sp.exp(expr), [t, 0, x])*sp.exp(-x**2)/(n*sp.sqrt(sp.pi))
expr = expr.simplify()
sp.print_latex(expr)
display(expr)

expr = expr.rewrite(sp.erfc)
expr = expr.expand().factor().simplify()
display(expr)
expr = erfc_to_erfcx(expr).expand()
expr = sp.Add(*map(lambda x: x.simplify(), expr.args))
sp.print_latex(expr)
display(expr)

In [ ]:
print(expr.args[0].args[-1].args[0].simplify())

In [ ]:
expr = sp.exp(-a**2*b**2/c + 2*a**2*b*h*n/c - a**2*h**2*n**2/c - b**2 + 2*b*h*n - h**2*n**2 - x**2)
expr = expr.diff(n)
display(expr)
print(sp.solve(expr, n))

print(sp.expand(expr.args[0]/(2*h)))

### Численная проверка

In [ ]:
hi, hj, lij, lik = 1.1, 1.4, 0.9, 1.6

print(Z3num.calc_Z3_integrate(hi, hj, hi, lij, lij, lik))
print(2*(2*np.pi)**3/np.sqrt(hi*hj*lij*lik)*Z3num.cF3_symmetrical(hi, hj, lij, lik))
print(Z3num.calc_from_coeffs(hi, hj, lij, lik).Z3_norm*np.exp(hj+lik+2*(hi+lij)))

In [ ]:
def get_diffs(expr, args, n=1, func=None):
    diffs = []
    for arg in args:
        _expr = expr.diff(arg, n)
        if func is not None:
            _expr = func(_expr)
        diffs.append(sp.lambdify(args, _expr, Z3sym.num_module))
    return diffs

In [ ]:
def calc_numerical_diffs(func, args, n=1):
    diffs = np.zeros_like(args)
    for i, arg in enumerate(args):
        direction = [0] * len(args)
        direction[i] = 1
        diffs[i] = nd.directionaldiff(func, args, direction, n=n)
    return diffs


def calc_exprs(exprs, args):
    return list(map(lambda expr: expr(*args), exprs))


def compare_exprs(exprs_1, exprs_2):
    for i, expr in enumerate(exprs_1):
        print(expr)
        print(exprs_2[i])
        print()

In [ ]:
x, a, b = sp.symbols("x a b")

f_F3_diffs = get_diffs(F3(x, a, b), [x, a, b])

args = 1.4, -6, 3
compare_exprs(
    calc_numerical_diffs(lambda args: Z3num.F3(*args), args),
    calc_exprs(f_F3_diffs, args)
)

In [ ]:
f_F3_degenerate_diffs = get_diffs(F3_degenerate(x, a, b, 0, 0), [x, a, b], func=evaluate_F3_degenerate)

args = 1.4, 1, 0
compare_exprs(
    calc_numerical_diffs(lambda args: Z3num.F3(*args), args),
    calc_exprs(f_F3_degenerate_diffs, args)
)

In [ ]:
expr = F3_degenerate(x, a, b, 0, 0).diff(x, x, a)
expr = evaluate_F3_degenerate(expr)
f_expr = sp.lambdify([x, a, b], expr, Z3sym.num_module)

args = 1.4, 1, 0
f = lambda args: nd.directionaldiff(lambda args: Z3num.F3(*args), args, [1, 0, 0], n=2)
print(nd.directionaldiff(f, args, [0, 1, 0], n=1))
print(f_expr(*args))

In [ ]:
hi, hj, lij, lik = Z3sym.hi, Z3sym.hj, Z3sym.lij, Z3sym.lik

f_E_123 = sp.lambdify((hi, hj, lij, lik ), EFunction(hi, hj, lij, lik , 1, 1, 1), Z3sym.num_module)
f_E_123_diffs = get_diffs(EFunction(hi, hj, lij, lik , 1, 1, 1), (hi, hj, lij, lik ))

args = 1.1, 1.6, 0.9, 1.2
compare_exprs(
    calc_numerical_diffs(lambda x: f_E_123(*x), args),
    calc_exprs(f_E_123_diffs, args)
)

In [ ]:
f_expr = sp.lambdify((hi, hj, lij, lik), EFunction(hi, hj, lij, lik, 1, 1, 1).rewrite(sp.exp), Z3sym.num_module)

args = 1.1, 1.9, 0.9, 1.2
print(f_expr(*args))
print(f_E_123(*args))

In [ ]:
f_F_123 = sp.lambdify((hi, hj, lij, lik), FFunction(hi, hj, lij, lik, 1, 1, 1), Z3sym.num_module)
f_F_123_diffs = get_diffs(FFunction(hi, hj, lij, lik, 1, 1, 1), (hi, hj, lij, lik))

args = 1.0, 1.6, 0.9, 1.2
compare_exprs(
    calc_numerical_diffs(lambda x: f_F_123(*x), args),
    calc_exprs(f_F_123_diffs, args)
)

In [ ]:
f_expr = sp.lambdify((hi, hj, lij, lik), FFunction(hi, hj, lij, lik, 1, 1, 1).rewrite(sp.exp), Z3sym.num_module)

args = 1.1, 1.9, 0.9, 1.5
print(f_expr(*args))
print(f_F_123(*args))

In [ ]:
f_F_tilde_123 = sp.lambdify((hi, hj, lij, lik), FTildeFunction(hi, hj, lij, lik, 1, 1, 1), Z3sym.num_module)
f_F_tilde_123_diffs = get_diffs(FTildeFunction(hi, hj, lij, lik, 1, 1, 1), (hi, hj, lij, lik))

args = 1.1, 1.6, 0.9, 1.3
compare_exprs(
    calc_numerical_diffs(lambda x: f_F_tilde_123(*x), args),
    calc_exprs(f_F_tilde_123_diffs, args)
)

In [ ]:
f_expr = sp.lambdify((hi, hj, lij, lik), FTildeFunction(hi, hj, lij, lik, 1, 1, 1), Z3sym.num_module)

args = 1.1, 1.9, 0.9, 1.9
print(f_expr(*args))
print(f_F_tilde_123(*args))

In [ ]:
f_cF3_diffs = get_diffs(cF3(hi, hj, lij, lik), (hi, hj, lij, lik))

args = 1.1, 1.6, 0.9, 1.2
compare_exprs(
    calc_numerical_diffs(lambda x: Z3num.cF3_symmetrical(*x), args),
    calc_exprs(f_cF3_diffs, args)
)

In [ ]:
f_expr = sp.lambdify((hi, hj, lij, lik),cF3(hi, hj, lij, lik).rewrite(sp.exp), Z3sym.num_module)

args = 1.1, 1.9, 0.9, 1.6
print(f_expr(*args))
print(Z3num.cF3_symmetrical(*args))

In [ ]:
f_cF3_degenerate_diffs = get_diffs(cF3_degenerate(hi, hj, lij, lik, *(4*[0])), (hi, hj, lij, lik), n=1, func=evaluate_cF3_degenerate)

args = 1.1, 1.9, 1.9, 1.1
compare_exprs(
    calc_numerical_diffs(lambda x: Z3num.cF3_symmetrical(*x), args, n=1),
    calc_exprs(f_cF3_degenerate_diffs, args)
)

## Моменты $\cal Z^{(3)}$

### Определения

In [ ]:
def get_line_Matrix(expr, like_terms):
    terms = sp.collect(expr, like_terms, evaluate=False)
    line = []
    for term in like_terms + [1]:
        if term in terms:
            if term == 1:
                line.append(-terms[term].factor())
            else:
                line.append(terms[term].factor())
        else:
            line.append(0)
    return line


### Общий случай

#### Символьные вычисления

In [ ]:
mi, mj, eta_ij, eta_ik = Z3sym.mi, Z3sym.mj, Z3sym.eta_ij, Z3sym.eta_ik

In [ ]:
Z3 = cF3(hi, hj, lij, lik)/sp.sqrt(hi*hj*lij*lik)

expr_mi = sp.log(Z3).diff(hi)/2
expr_mi = sp.Add(*map(sp.factor, expr_mi.expand().args))

expr_mj = sp.log(Z3).diff(hj)
expr_mj = sp.Add(*map(sp.factor, expr_mj.expand().args))

expr_eta_ij = sp.log(Z3).diff(lij)/2
expr_eta_ij = sp.Add(*map(sp.factor, expr_eta_ij.expand().args))

expr_eta_ik = sp.log(Z3).diff(lik)
expr_eta_ik = sp.Add(*map(sp.factor, expr_eta_ik.expand().args))

In [ ]:
n, d = sp.fraction(expr_mi.factor())
_expr_mi = n-d*mi

n, d = sp.fraction(expr_mj.factor())
_expr_mj = n-d*mj

n, d = sp.fraction(expr_eta_ij.factor())
_expr_eta_ij = n-d*eta_ij

n, d = sp.fraction(expr_eta_ik.factor())
_expr_eta_ik = n-d*eta_ik

In [ ]:
vector_system_moments = [
    cF3(hi, hj, lij, lik),
    FTildeFunction(hi, hj, lij, lik, 1, 1, 1),
    FTildeFunction(hi, hj, lij, lik, 1, 1, 0),
    FFunction(hi, hj, lij, lik, 1, 0, 1),
]

line1 = get_line_Matrix(_expr_mi, vector_system_moments)
line2 = get_line_Matrix(_expr_mj, vector_system_moments)
line3 = get_line_Matrix(_expr_eta_ij, vector_system_moments)
line4 = get_line_Matrix(_expr_eta_ik, vector_system_moments)

In [ ]:
M = sp.Matrix([line1, line2, line3, line4])
M[0, :] /= sp.sqrt(lij)*hi*sp.sqrt(hi)/sp.sqrt(hj)
M[1, :] /= sp.sqrt(lij)*sp.sqrt(hi)
M[2, :] /= sp.sqrt(lij)*lij*sp.sqrt(hi)/sp.sqrt(hj)
M[3, :] /= sp.sqrt(lik)/sp.sqrt(hj)

M[0, :] += M[1, :]
M[1, :] += M[2, :]
M[2, :] += M[0, :]/2
M[2, :] -= M[1, :]
M[2, :] += lik*hj*M[3, :]
M[3, :] -= M[2, :]/(lik*hj)
M = M.expand()

M = sp.Matrix(4, 5, lambda i, j: M[i, j].factor())
M[0, :] /= (lij*hi-lik*hj)*hi*sp.sqrt(hj)*2
M[1, :] /= (lij*hi-lik*hj)*lij*sp.sqrt(hj)*2
M[2, :] /= (lij*hi-lik*hj)*sp.sqrt(hj)
M[3, :] *= lik*sp.sqrt(hj)

M[2, :] -= M[0, :]*hi
M[3, :] += M[0, :]*hi**2*lij
M[3, :] += M[1, :]*hj*lij*lik
M[2, :] += M[3, :]/(hi*lij)
M[3, :] += M[2, :]/(lik*hj)*hi**2*lij**2
M = M.expand()

M = sp.Matrix(4, 5, lambda i, j: M[i, j].factor())
M[0, :] /= -2
M[1, :] /= -2
M[2, :] /= -2*lik*hj
M[3, :] /= -2*lij*hi

det_system_moments = sp.Symbol("\\Delta")
det_system_moments_value = M[3, 0]
M[3, 0] = det_system_moments

display(M)

In [ ]:
det_system_moments_value.collect([mi, mj, eta_ij, eta_ik], sp.factor)

In [ ]:
result = sp.solve_linear_system(M, *vector_system_moments)

value_system_moments = []
for key in vector_system_moments:
    value_system_moments.append(result[key].factor())

for i in range(4):
    display(sp.Eq(vector_system_moments[i], value_system_moments[i]))
for i in range(4):
    sp.print_latex(sp.Eq(vector_system_moments[i], value_system_moments[i]))

display(sp.Eq(det_system_moments, det_system_moments_value))
sp.print_latex(sp.Eq(det_system_moments, det_system_moments_value))

In [ ]:
subs_to_moments = dict(zip(vector_system_moments, value_system_moments))
subs_det_system_moments = {det_system_moments: det_system_moments_value}

In [ ]:
expr = expr_eta_ik
expr.subs(subs_to_moments).subs(subs_det_system_moments).factor()

In [ ]:
expr_Q_star = Z3.diff(lij, 2)/(2*Z3) - expr_eta_ik - 1
expr_Q_star = sp.Add(*map(sp.factor, expr_Q_star.expand().args))
# expr_Q_star

In [ ]:
expr_diff_hi_lij = Z3.diff(hi, lij)/(Z3)
expr_diff_hi_lij = sp.Add(*map(sp.factor, expr_diff_hi_lij.expand().args))
expr_diff_hi_lij = expr_diff_hi_lij.subs(subs_to_moments)

expr_mj_eta_ij = Z3.diff(hj, lij)/(2*Z3)
expr_mj_eta_ij = sp.Add(*map(sp.factor, expr_mj_eta_ij.expand().args))
expr_mj_eta_ij = expr_mj_eta_ij.subs(subs_to_moments)

expr_mi_eta_ik = Z3.diff(hi, lik)/(2*Z3)
expr_mi_eta_ik = sp.Add(*map(sp.factor, expr_mi_eta_ik.expand().args))
expr_mi_eta_ik = expr_mi_eta_ik.subs(subs_to_moments)

expr_eta_ik_2 = Z3.diff(lik, 2)/(Z3)
expr_eta_ik_2 = sp.Add(*map(sp.factor, expr_eta_ik_2.expand().args))
expr_eta_ik_2 = expr_eta_ik_2.subs(subs_to_moments)

In [ ]:
eta_ik_2, mj_eta_ij, mi_eta_ik = Z3sym.eta_ik_2, Z3sym.mj_eta_ij, Z3sym.mi_eta_ik

In [ ]:
factor = (lij*hi-lik*hj)**2

expr_diff_hi_lij_dummy = sp.Dummy()
expr1 = (expr_diff_hi_lij.factor()*factor).expand() - factor*expr_diff_hi_lij_dummy
expr2 = (expr_mj_eta_ij.factor()*factor).expand() - factor*mj_eta_ij
expr3 = (expr_mi_eta_ik.factor()*factor).expand() - factor*mi_eta_ik
expr4 = (expr_eta_ik_2.factor()*factor).expand() - factor*eta_ik_2

In [ ]:
vector_system_moments_2 = [
    EFunction(hi, hj, lij, lik, 0, 0, 1)/EFunction(hi, hj, lij, lik, 1, 1, 1),
    EFunction(hi, hj, lij, lik, 1, 0, 0)/EFunction(hi, hj, lij, lik, 1, 1, 1),
    EFunction(hi, hj, lij, lik, 1, 1, 0)/EFunction(hi, hj, lij, lik, 1, 1, 1),
    EFunction(hi, hj, lij, lik, 1, 0, 1)/EFunction(hi, hj, lij, lik, 1, 1, 1),
]

line1 = get_line_Matrix(expr1, vector_system_moments_2)
line2 = get_line_Matrix(expr2, vector_system_moments_2)
line3 = get_line_Matrix(expr3, vector_system_moments_2)
line4 = get_line_Matrix(expr4, vector_system_moments_2)

In [ ]:
M = sp.Matrix([line1, line2, line3, line4])
M = M.expand()

M[0, :] += M[1, :]*2/hi*hj
M[1, :] -= M[2, :]*hi**2/hj**2
M[2, :] += M[3, :]/2/hi*lik
M[3, :] += M[2, :]*2*hi/lik

M = M.expand()

display(M)

In [ ]:
value_system_moments_2 = []

result = (M[:, :-1]).solve(M[:, -1])
for item in result:
    value_system_moments_2.append(item.factor())

# for i in range(4):
#     display(sp.Eq(vector_system_moments_2[i], value_system_moments_2[i]))
# for i in range(4):
#     sp.print_latex(sp.Eq(vector_system_moments_2[i], value_system_moments_2[i]))

In [ ]:
subs_to_moments_2 = dict(zip(vector_system_moments_2, value_system_moments_2))

In [ ]:
expr_Q_star_from_m_j_eta_ij = expr_Q_star.subs(subs_to_moments).subs(subs_to_moments_2).subs(subs_det_system_moments).factor().expand()
display(expr_Q_star_from_m_j_eta_ij)
sp.print_latex((lij*expr_Q_star_from_m_j_eta_ij).expand())

In [ ]:
subs_to_Q_star = {lij: sp.solve(expr_Q_star_from_m_j_eta_ij-Z3sym.Q_star, lij)[0]}

In [ ]:
expr_mjh2 = Z3.diff(hj, 2)/(Z3)
expr_mjh2 = sp.Add(*map(sp.factor, expr_mjh2.expand().args))
expr = expr_mjh2.subs(subs_to_moments).subs(subs_to_moments_2).subs(subs_det_system_moments).factor().expand()

mjh2 = Z3sym.mjh2
display(sp.Eq(mjh2, expr))
sp.print_latex(expr)
 
expr_Q_star_from_mjh2 = expr_Q_star_from_m_j_eta_ij.replace(mj_eta_ij, sp.solve(expr-mjh2, mj_eta_ij)[0]).expand()
display(expr_Q_star_from_mjh2)
sp.print_latex((lij*expr_Q_star_from_mjh2).expand())

_expr_mjh2 = expr

In [ ]:
expr_mj_eta_ik = Z3.diff(hj, lik)/(Z3)
expr_mj_eta_ik = sp.Add(*map(sp.factor, expr_mj_eta_ik.expand().args))
expr = expr_mj_eta_ik.subs(subs_to_moments).subs(subs_to_moments_2).subs(subs_det_system_moments).factor().expand()

mj_eta_ik = Z3sym.mj_eta_ik
subs_to_mj_eta_ik = {expr_diff_hi_lij_dummy: sp.solve(expr-mj_eta_ik, expr_diff_hi_lij_dummy)[0]}

expr

In [ ]:
M = sp.zeros(4, 4)

for i, moment in zip(range(4), ["mi", "mj", "eta_ij", "eta_ik"]):
    for j, arg, name_arg in zip(range(4), [hi, hj, lij, lik], ["h_i", "h_j", "l_ij", "l_ik"]):
        # expr = expr_moment.diff(arg).subs(subs_to_moments)
        # expr = sp.Add(*map(sp.factor, expr.expand().args))
        # expr = expr.subs(subs_to_moments_2).subs(subs_det_system_moments)
        # expr = expr.factor().expand()
        # expr = expr.subs(subs_to_mj_eta_ik).expand()
        # print(f"double {moment}_{name_arg} = ", get_ccode(optimize_moment(expr).subs(subs)), ";")
        M[i, j] = sp.Symbol(f"{moment}_{name_arg}")
        # display(sp.Eq(sp.Derivative(moment, arg), expr))
    print()

In [ ]:
x1, x2, x3, x4 = sp.symbols("x1 x2 x3 x4")
for i, item, name_arg in zip(range(4), M.cramer_solve(sp.Matrix([x1, x2, x3, x4])), ["h_i", "h_j", "l_ij", "l_ik"]):
    n, d = sp.fraction(item)
    if i == 0:
        print(f"double D = {d};")
    print(f"delta_{name_arg} = _D * ({n});")

#### Численная проверка

In [ ]:
def calc_Z3_symmetrical_integrate(args):
    hi, hj, lij, lik = args
    return Z3num.calc_Z3_integrate(hi, hj, hi, lij, lij, lik)

In [ ]:
args_coeffs = Z3sym.hi, Z3sym.hj, Z3sym.lij, Z3sym.lik
args_moments = Z3sym.hi, Z3sym.hj, Z3sym.lij, Z3sym.lik, mi, mj, eta_ij, eta_ik

In [ ]:
f_expr_m_i = sp.lambdify(args_coeffs, expr_mi, Z3sym.num_module_norm)
f_expr_m_j = sp.lambdify(args_coeffs, expr_mj, Z3sym.num_module_norm)
f_expr_eta_ij = sp.lambdify(args_coeffs, expr_eta_ij, Z3sym.num_module_norm)
f_expr_eta_ik = sp.lambdify(args_coeffs, expr_eta_ik, Z3sym.num_module_norm)

args = 1.4, 1.1, 0.9, 1.2
compare_exprs(
    calc_numerical_diffs(calc_Z3_symmetrical_integrate, args)/(np.asarray([2, 1, 2, 1])*calc_Z3_symmetrical_integrate(args)),
    calc_exprs([f_expr_m_i, f_expr_m_j, f_expr_eta_ij, f_expr_eta_ik], args)
)

In [ ]:
z3 = Z3num.calc_from_coeffs(*args)

print(f_expr_m_i(*args))
print(z3.mi)
print()
print(f_expr_m_j(*args))
print(z3.mj)
print()
print(f_expr_eta_ij(*args))
print(z3.eta_ij)
print()
print(f_expr_eta_ik(*args))
print(z3.eta_ik)
print()

In [ ]:
args = 2, 1, 1.1, 1.4
moments = f_expr_m_i(*args), f_expr_m_j(*args), f_expr_eta_ij(*args), f_expr_eta_ik(*args)

for i in range(4):
    left = vector_system_moments[i].subs(subs_det_system_moments)
    f_left = sp.lambdify(args_moments, left, Z3sym.num_module_norm)
    print(f_left(*args, *moments))

    right = value_system_moments[i].subs(subs_det_system_moments)
    f_right = sp.lambdify(args_moments, right, Z3sym.num_module_norm)
    print(f_right(*args, *moments))

    print()

In [ ]:
f_expr_Q_star = sp.lambdify(args_coeffs, expr_Q_star, Z3sym.num_module_norm)

args = 0.2073412, 0.13806503, 1.26262682, 0.15322752
Q_star = nd.directionaldiff(calc_Z3_symmetrical_integrate, args, [0, 0, 1, 0], n=2)/(2*calc_Z3_symmetrical_integrate(args)) - f_expr_eta_ik(*args) - 1
print(Q_star)
print(f_expr_Q_star(*args))

moments = f_expr_m_i(*args), f_expr_m_j(*args), f_expr_eta_ij(*args), f_expr_eta_ik(*args)

f_expr_Q_star_from_m_j_eta_ij = sp.lambdify(list(args_moments) + [mj_eta_ij], expr_Q_star_from_m_j_eta_ij, Z3sym.num_module_norm)
f_expr_m_j_eta_ij = sp.lambdify(args_moments, expr_mj_eta_ij.subs(subs_det_system_moments), Z3sym.num_module_norm)
print(f_expr_Q_star_from_m_j_eta_ij(*args, *moments, f_expr_m_j_eta_ij(*args, *moments)))

f_expr_Q_star_from_m_j_par_2 = sp.lambdify(list(args_moments) + [mjh2], expr_Q_star_from_mjh2, Z3sym.num_module_norm)
f_expr_m_j_par_2 = sp.lambdify(args_moments, expr_mjh2.subs(subs_det_system_moments), Z3sym.num_module_norm)
print(f_expr_Q_star_from_m_j_par_2(*args, *moments, f_expr_m_j_par_2(*args, *moments)))

z3 = Z3num.calc_from_coeffs(*args)
print(z3.Q_star)

In [ ]:
print(f_expr_m_j_eta_ij(*args, *moments))
print(z3.mj_eta_ij)

In [ ]:
print(f_expr_m_j_par_2(*args, *moments))
print(z3.mjh2)

In [ ]:
hi, hj, lij, lik = 1.1, 1.4, 0.9, 1.6
nd.Gradient(lambda args: Z3num.calc_Z3_integrate(*args))((hi, hj, hi, lij, lij, lik))/Z3num.calc_Z3_integrate(hi, hj, hi, lij, lij, lik)

In [ ]:
hi, hj, lij, lik = 1.1, 1.4, 0.9, 1.6
hessian = nd.Hessian(lambda args: Z3num.calc_Z3_integrate(*args))((hi, hj, hi, lij, lij, lik))/Z3num.calc_Z3_integrate(hi, hj, hi, lij, lij, lik)
print(hessian)

### Вырожденный случай

#### Символьные вычисления

In [ ]:
hi, hj, lij, lik = Z3sym.hi, Z3sym.hj, Z3sym.lij, Z3sym.lik 

In [ ]:
_expr = cF3_degenerate(hi, hj, lij, lik, *(4*[0]))

subs_to_cF3Function_degenerate = {
    EFunction(hi, hj, lij, lik, 1, 1, 1): sp.solve(_expr-evaluate_cF3_degenerate(_expr), EFunction(hi, hj, lij, lik, 1, 1, 1))[0]
}

In [ ]:
Z3_degenerate_expr = cF3_degenerate(hi, hj, lij, lik, *(4*[0]))/sp.sqrt(hi*hj*lij*lik)

In [ ]:
expr_m_i_degenerate = sp.log(Z3_degenerate_expr).diff(hi)/2
expr_m_i_degenerate = expr_m_i_degenerate.expand()
expr_m_i_degenerate = evaluate_cF3_degenerate(expr_m_i_degenerate)
expr_m_i_degenerate = expr_m_i_degenerate.subs(subs_to_cF3Function_degenerate).expand()

expr_m_j_degenerate = sp.log(Z3_degenerate_expr).diff(hj)
expr_m_j_degenerate = expr_m_j_degenerate.expand()
expr_m_j_degenerate = evaluate_cF3_degenerate(expr_m_j_degenerate)
expr_m_j_degenerate = expr_m_j_degenerate.subs(subs_to_cF3Function_degenerate).expand()

expr_eta_ij_degenerate = sp.log(Z3_degenerate_expr).diff(lij)/2
expr_eta_ij_degenerate = expr_eta_ij_degenerate.expand()
expr_eta_ij_degenerate = evaluate_cF3_degenerate(expr_eta_ij_degenerate)
expr_eta_ij_degenerate = expr_eta_ij_degenerate.subs(subs_to_cF3Function_degenerate).expand()

expr_eta_ik_degenerate = sp.log(Z3_degenerate_expr).diff(lik)
expr_eta_ik_degenerate = expr_eta_ik_degenerate.expand()
expr_eta_ik_degenerate = evaluate_cF3_degenerate(expr_eta_ik_degenerate)
expr_eta_ik_degenerate = expr_eta_ik_degenerate.subs(subs_to_cF3Function_degenerate).expand()

In [ ]:
n, d = sp.fraction(expr_m_i_degenerate.factor())
_expr_mi = (n-d*mi).expand()

n, d = sp.fraction(expr_m_j_degenerate.factor())
_expr_mj = (n-d*mj).expand()

n, d = sp.fraction(expr_eta_ij_degenerate.factor())
_expr_eta_1 = (n-d*eta_ij).expand()

n, d = sp.fraction(expr_eta_ik_degenerate.factor())
_expr_eta_2 = (n-d*eta_ik).expand()

In [ ]:
vector_system_moments_degenerate = [
    FFunction(hi, hj, lij, lik, 1, 1, 0),
    FFunction(hi, hj, lij, lik, 1, 0, 1),
    FFunction(hi, hj, lij, lik, 1, 1, 1),
    cF3_degenerate(hi, hj, lij, lik, *(4*[0])),
]

line1 = get_line_Matrix(_expr_mi, vector_system_moments_degenerate)
line2 = get_line_Matrix(_expr_mj, vector_system_moments_degenerate)
line3 = get_line_Matrix(_expr_eta_1, vector_system_moments_degenerate)
line4 = get_line_Matrix(_expr_eta_2, vector_system_moments_degenerate)

In [ ]:
M = sp.Matrix([line1, line2, line3, line4])
M = M.replace(lik, lij*hi/hj)

M[0, :] /= (2*hi**3*hj**2*lij**2)
M[1, :] /= (2*hi**2*hj**3*lij)
M[2, :] /= (2*hi*hj**2*lij**2)
M[3, :] /= (2*hi**2*lij**2)

M[0, :] += M[1, :]/lij
M[1, :] += M[2, :]/hi
M[2, :] += M[3, :]/hj

M = M.expand()

A_system_moments_degenerate = sp.Symbol("A")
A_system_moments_degenerate_value = (M[3, 4]/(lij*hi*hj)).expand()
M[3, 4] = A_system_moments_degenerate*(lij*hi*hj)

display(M)
display(A_system_moments_degenerate_value)
sp.print_latex(A_system_moments_degenerate_value)

In [ ]:
vector_system_moments_degenerate = list(map(lambda item: item.replace(lik, lij*hi/hj), vector_system_moments_degenerate))

result = sp.solve_linear_system(M, *vector_system_moments_degenerate)

value_system_moments_degenerate = []
for key in vector_system_moments_degenerate:
    value_system_moments_degenerate.append(result[key].factor())

n, d = sp.fraction(value_system_moments_degenerate[0])
det_system_moments_degenerate_value = d/2
det_system_moments_degenerate = sp.Symbol("\\Delta")
subs_det_system_moments_degenerate = {det_system_moments_degenerate: det_system_moments_degenerate_value}

value_system_moments_degenerate = list(map(lambda item: item.subs(det_system_moments_degenerate_value, det_system_moments_degenerate), value_system_moments_degenerate))

for i in range(4):
    display(sp.Eq(vector_system_moments_degenerate[i], value_system_moments_degenerate[i]))
for i in range(4):
    sp.print_latex(sp.Eq(vector_system_moments_degenerate[i], value_system_moments_degenerate[i]))

display(sp.Eq(det_system_moments_degenerate, det_system_moments_degenerate_value))
sp.print_latex(sp.Eq(det_system_moments_degenerate, det_system_moments_degenerate_value))

In [ ]:
expr = det_system_moments_degenerate_value.collect([mi, mj, eta_ij, eta_ik], sp.factor)
display(expr)
sp.print_latex(expr)
det_system_moments_degenerate_value.subs({hj: hi, mj: mi, eta_ik: eta_ij})

In [ ]:
subs_to_A_degenerate = {EFunction(hi, hj, lij, lij*hi/hj, 0, 0, 1): sp.solve(A_system_moments_degenerate_value-A_system_moments_degenerate, EFunction(hi, hj, lij, lij*hi/hj, 0, 0, 1))[0]}
subs_to_moments_degenerate = dict(zip(vector_system_moments_degenerate, value_system_moments_degenerate))

In [ ]:
expr = expr_eta_ik_degenerate.replace(lik, lij*hi/hj)
expr.subs(subs_to_A_degenerate).subs(subs_to_moments_degenerate).subs(subs_det_system_moments_degenerate).factor()

In [ ]:
def calc_degenerate(expr):
    expr = expr.expand()
    expr = evaluate_cF3_degenerate(expr)
    expr = expr.subs(subs_to_cF3Function_degenerate).expand()
    expr = expr.replace(lik, lij*hi/hj)
    expr = expr.subs(subs_to_A_degenerate).subs(subs_to_moments_degenerate).expand()
    return expr

In [ ]:
expr_Q_star_degenerate = calc_degenerate(Z3_degenerate_expr.diff(lij, 2)/(2*Z3_degenerate_expr) - eta_ik - 1)

# expr_Q_star_degenerate

In [ ]:
expr_diff_h_i_l_ij_degenerate = calc_degenerate(Z3_degenerate_expr.diff(hi, lij)/(Z3_degenerate_expr))
expr_m_j_eta_ij_degenerate = calc_degenerate(Z3_degenerate_expr.diff(hj, lij)/(2*Z3_degenerate_expr))
expr_m_i_eta_ik_degenerate = calc_degenerate(Z3_degenerate_expr.diff(hi, lik)/(2*Z3_degenerate_expr))
expr_eta_ik_2_degenerate = calc_degenerate(Z3_degenerate_expr.diff(lik, 2)/(Z3_degenerate_expr))

In [ ]:
vector_system_moments_2_degenerate = [
    EFunction(hi, hj, lij, lij*hi/hj, 0, 0, 0)/A_system_moments_degenerate,
    EFunction(hi, hj, lij, lij*hi/hj, 1, 1, 0)/A_system_moments_degenerate,
    EFunction(hi, hj, lij, lij*hi/hj, 1, 0, 1)/A_system_moments_degenerate,
    EFunction(hi, hj, lij, lij*hi/hj, 1, 0, 0)/A_system_moments_degenerate
]

line1 = get_line_Matrix(expr_diff_h_i_l_ij_degenerate-sp.Dummy(), vector_system_moments_2_degenerate)
line2 = get_line_Matrix(expr_m_j_eta_ij_degenerate-mj_eta_ij, vector_system_moments_2_degenerate)
line3 = get_line_Matrix(expr_m_i_eta_ik_degenerate-mi_eta_ik, vector_system_moments_2_degenerate)
line4 = get_line_Matrix(expr_eta_ik_2_degenerate-eta_ik_2, vector_system_moments_2_degenerate)

In [ ]:
M = sp.Matrix([line1, line2, line3, line4])

M[0, :] += M[1, :]*2/hi*hj
M[1, :] -= M[2, :]/hj**2*hi**2
M[2, :] += M[3, :]*lij/hj/2

M[1, :] -= M[0, :]/hj*hi/2
M[2, :] += M[1, :]*hj**2/2/hi**2

M = M.expand()

display(M)

In [ ]:
value_system_moments_2_degenerate = []

value_system_moments_2_degenerate.append((M[0, -1]/M[0, 1]).factor())
value_system_moments_2_degenerate.append((M[1, -1]/M[1, 2]).factor())
value_system_moments_2_degenerate.append((M[2, -1]/M[2, 3]).factor())

expr = sum([-M[-1, i+1]*value_system_moments_2_degenerate[i] for i in range(3)])+M[-1, -1]
value_system_moments_2_degenerate.insert(0, (expr/M[-1, 0]).factor())

# for i in range(4):
#     display(sp.Eq(vector_system_moments_2_degenerate[i], value_system_moments_2_degenerate[i]))
# for i in range(4):
#     sp.print_latex(sp.Eq(vector_system_moments_2_degenerate[i], value_system_moments_2_degenerate[i]))

In [ ]:
subs_to_moments_2_degenerate = dict(zip(vector_system_moments_2_degenerate, value_system_moments_2_degenerate))

In [ ]:
expr_eta_ik_2_degenerate.subs(subs_to_moments_2_degenerate).factor()

In [ ]:
expr_Q_star_degenerate_from_m_j_eta_ij = expr_Q_star_degenerate.subs(subs_to_moments_2_degenerate).subs(subs_det_system_moments_degenerate).factor().expand()
expr_Q_star_degenerate_from_m_j_eta_ij

In [ ]:
expr_m_j_par_2_degenerate = calc_degenerate(Z3_degenerate_expr.diff(hj, 2)/(Z3_degenerate_expr))
expr = expr_m_j_par_2_degenerate.subs(subs_to_moments_2_degenerate).subs(subs_det_system_moments_degenerate).factor().expand()

display(sp.Eq(mjh2, expr))

In [ ]:
expr_m_j_eta_ik_degenerate = calc_degenerate(Z3_degenerate_expr.diff(hj, lik)/(Z3_degenerate_expr))
expr = expr_m_j_eta_ik_degenerate.subs(subs_to_moments_2_degenerate).subs(subs_det_system_moments_degenerate).factor().expand()
expr

#### Проверка

In [ ]:
to_cF3 = lambda *args: cF3(*args[:4])

f_expr_m_i_degenerate = sp.lambdify(args_coeffs, expr_m_i_degenerate.replace(cF3_degenerate, to_cF3), Z3sym.num_module_norm)
f_expr_m_j_degenerate = sp.lambdify(args_coeffs, expr_m_j_degenerate.replace(cF3_degenerate, to_cF3), Z3sym.num_module_norm)
f_expr_eta_ij_degenerate = sp.lambdify(args_coeffs, expr_eta_ij_degenerate.replace(cF3_degenerate, to_cF3), Z3sym.num_module_norm)
f_expr_eta_ik_degenerate = sp.lambdify(args_coeffs, expr_eta_ik_degenerate.replace(cF3_degenerate, to_cF3), Z3sym.num_module_norm)

args = 1, 2, 2., 1
compare_exprs(
    calc_numerical_diffs(calc_Z3_symmetrical_integrate, args)/(np.asarray([2, 1, 2, 1])*calc_Z3_symmetrical_integrate(args)),
    calc_exprs([f_expr_m_i_degenerate, f_expr_m_j_degenerate, f_expr_eta_ij_degenerate, f_expr_eta_ik_degenerate], args)
)

In [ ]:
z3 = Z3num.calc_from_coeffs(*args)

print(f_expr_m_i_degenerate(*args))
print(z3.mi)
print()
print(f_expr_m_j_degenerate(*args))
print(z3.mj)
print()
print(f_expr_eta_ij_degenerate(*args))
print(z3.eta_ij)
print()
print(f_expr_eta_ik_degenerate(*args))
print(z3.eta_ik)
print()

In [ ]:
args_moments = hi, hj, lij, lik, mi, mj, eta_ij, eta_ik

In [ ]:
args = 2, 1, 1.1
args = list(args) + [args[2]*args[0]/args[1]]
moments = f_expr_m_i_degenerate(*args), f_expr_m_j_degenerate(*args), f_expr_eta_ij_degenerate(*args), f_expr_eta_ik_degenerate(*args)

for i in range(4):
    left = vector_system_moments_degenerate[i].replace(cF3_degenerate, to_cF3).subs(subs_det_system_moments_degenerate)
    f_left = sp.lambdify(args_moments, left, Z3sym.num_module_norm)
    print(f_left(*args, *moments))

    right = value_system_moments_degenerate[i].replace(A_system_moments_degenerate, A_system_moments_degenerate_value).subs(subs_det_system_moments_degenerate)
    # right = A_system_moments_degenerate_value
    f_right = sp.lambdify(args_moments, right, Z3sym.num_module_norm)
    print(f_right(*args, *moments))

    print()

### Вырожденный случай с $h_1 = h_2$ и $\lambda_1 = \lambda_2$

#### Символьные вычисления

In [ ]:
class EFunction_spec_1(EFunction):
    @classmethod
    def eval(cls, *args):
        arg = args[4] + args[5] + args[6]

        if arg == 1 and not args[4]:
            return EFunction_spec_1(*args[:4], 1, 0, 0)
        
        if arg == 2 and (not args[4] or not args[5]):
            return EFunction_spec_1(*args[:4], 1, 1, 0)


class FFunction_spec_1(FFunction):
    @classmethod
    def eval(cls, *args):
        arg = args[4] + args[5] + args[6]

        if arg == 1 and not args[4]:
            return FFunction_spec_1(*args[:4], 1, 0, 0)
        
        if arg == 2 and (not args[4] or not args[5]):
            return FFunction_spec_1(*args[:4], 1, 1, 0)

In [ ]:
h, l = sp.symbols("h \\lambda")
m = sp.Symbol("\\left<m\\right>")
eta = sp.Symbol("\\left<\eta\\right>")

In [ ]:
_expr = cF3_degenerate(h, h, l, l, *(4*[0]))

subs_to_cF3Function_degenerate_spec_1 = {
    EFunction_spec_1(h, h, l, l, 1, 1, 1): sp.solve(_expr-evaluate_cF3_degenerate(_expr).replace(EFunction, EFunction_spec_1).replace(FFunction, FFunction_spec_1), EFunction_spec_1(h, h, l, l, 1, 1, 1))[0]
}

In [ ]:
subs_to_cF3Function_degenerate_spec_1

In [ ]:
expr_m_i_degenerate_spec_1 = evaluate_cF3_degenerate(expr_m_i_degenerate).replace(EFunction, EFunction_spec_1).replace(FFunction, FFunction_spec_1)
expr_m_i_degenerate_spec_1 = expr_m_i_degenerate_spec_1.subs({hi: h, hj: h, lij: l, lik: l})

expr_m_j_degenerate_spec_1 = evaluate_cF3_degenerate(expr_m_j_degenerate).replace(EFunction, EFunction_spec_1).replace(FFunction, FFunction_spec_1)
expr_m_j_degenerate_spec_1 = expr_m_j_degenerate_spec_1.subs({hi: h, hj: h, lij: l, lik: l})

expr_eta_ij_degenerate_spec_1 = evaluate_cF3_degenerate(expr_eta_ij_degenerate).replace(EFunction, EFunction_spec_1).replace(FFunction, FFunction_spec_1)
expr_eta_ij_degenerate_spec_1 = expr_eta_ij_degenerate_spec_1.subs({hi: h, hj: h, lij: l, lik: l})

expr_eta_ik_degenerate_spec_1 = evaluate_cF3_degenerate(expr_eta_ik_degenerate).replace(EFunction, EFunction_spec_1).replace(FFunction, FFunction_spec_1)
expr_eta_ik_degenerate_spec_1 = expr_eta_ik_degenerate_spec_1.subs({hi: h, hj: h, lij: l, lik: l})

In [ ]:
expr_m_i_degenerate_spec_1.subs(subs_to_cF3Function_degenerate_spec_1)

In [ ]:
expr_eta_ik_degenerate_spec_1

In [ ]:
n, d = sp.fraction(expr_m_i_degenerate_spec_1.factor())
_expr_m = (n-d*m).expand()

n, d = sp.fraction(expr_eta_ij_degenerate_spec_1.factor())
_expr_eta_ij = (n-d*eta).expand()

In [ ]:
vector_system_moments_degenerate_spec_1 = [
    FFunction_spec_1(h, h, l, l, 1, 1, 1),
    FFunction_spec_1(h, h, l, l, 1, 1, 0),
]

line1 = get_line_Matrix(_expr_m, vector_system_moments_degenerate_spec_1)
line2 = get_line_Matrix(_expr_eta_ij, vector_system_moments_degenerate_spec_1)

In [ ]:
result = sp.solve_linear_system(sp.Matrix([line1, line2]), *vector_system_moments_degenerate_spec_1)

value_system_moments_degenerate_spec_1 = []
for key in vector_system_moments_degenerate_spec_1:
    value_system_moments_degenerate_spec_1.append(result[key].factor())

for i in range(2):
    display(sp.Eq(vector_system_moments_degenerate_spec_1[i], value_system_moments_degenerate_spec_1[i]))
for i in range(2):
    sp.print_latex(sp.Eq(vector_system_moments_degenerate_spec_1[i], value_system_moments_degenerate_spec_1[i]))

In [ ]:
subs_to_moments_degenerate_spec_1 = dict(zip(vector_system_moments_degenerate_spec_1, value_system_moments_degenerate_spec_1))

In [ ]:
expr = expr_eta_ik_degenerate_spec_1
expr.subs(subs_to_moments_degenerate_spec_1).factor()

In [ ]:
def calc_degenerate_spec_1(expr):
    expr = expr.expand()
    expr = evaluate_cF3_degenerate(expr)
    expr = expr.replace(EFunction, EFunction_spec_1).replace(FFunction, FFunction_spec_1)
    expr = expr.subs({hi: h, hj: h, lij: l, lik: l})
    expr = expr.subs(subs_to_moments_degenerate_spec_1).factor().expand()
    return expr

In [ ]:
expr_Q_star_degenerate_spec_1 = calc_degenerate_spec_1(Z3_degenerate_expr.diff(lij, 2)/(2*Z3_degenerate_expr) - eta - 1)
# expr_Q_star_degenerate_spec_1

In [ ]:
expr_eta_ik_2_degenerate_spec_1 = calc_degenerate_spec_1(Z3_degenerate_expr.diff(lik, 2)/Z3_degenerate_expr)
expr_m_j_eta_ik_degenerate_spec_1 = calc_degenerate_spec_1(Z3_degenerate_expr.diff(hj, lik)/Z3_degenerate_expr)
expr_m_j_eta_ij_degenerate_spec_1 = calc_degenerate_spec_1(Z3_degenerate_expr.diff(hj, lij)/(2*Z3_degenerate_expr))
expr_m_j_par_2_degenerate_spec_1 = calc_degenerate_spec_1(Z3_degenerate_expr.diff(hj, hj)/Z3_degenerate_expr)

In [ ]:
vector_system_moments_2_degenerate_spec_1 = [
    EFunction_spec_1(h, h, l, l, 1, 1, 0)/EFunction_spec_1(h, h, l, l, 1, 1, 1),
    EFunction_spec_1(h, h, l, l, 1, 0, 0)/EFunction_spec_1(h, h, l, l, 1, 1, 1),
    EFunction_spec_1(h, h, l, l, 0, 0, 0)/EFunction_spec_1(h, h, l, l, 1, 1, 1),
]

mj_eta_ik = sp.Symbol("\\left<m_j\\eta_{ik}\\right>")
line1 = get_line_Matrix(expr_eta_ik_2_degenerate_spec_1-eta_ik_2, vector_system_moments_2_degenerate_spec_1)
line2 = get_line_Matrix(expr_m_j_eta_ik_degenerate_spec_1-mj_eta_ik, vector_system_moments_2_degenerate_spec_1)
line3 = get_line_Matrix(expr_m_j_par_2_degenerate_spec_1-mjh2, vector_system_moments_2_degenerate_spec_1)

In [ ]:
M = sp.Matrix([line1, line2, line3])

M[0, :] *= 10*l**4
M[1, :] *= 10*l**3
M[2, :] *= 10*l**2*h**2

M[1, :] -= M[2, :]/h
M[0, :] -= M[2, :]
M[0, :] -= M[1, :]*h
M[2, :] -= M[0, :]/5*2
M[2, :] -= M[0, :]/5*2*h**2/l
M[2, :] -= M[1, :]/5*3*h*l
M[2, :] += M[1, :]/5*3*h
M[2, :] += M[1, :]/5/l*h**3

M = M.expand()

display(M)

In [ ]:
value_system_moments_2_degenerate_spec_1 = []

result = (M[:, :-1]).solve(M[:, -1])
for item in result:
    value_system_moments_2_degenerate_spec_1.append(item.factor())

for i in range(3):
    display(sp.Eq(vector_system_moments_2_degenerate_spec_1[i], value_system_moments_2_degenerate_spec_1[i]))
for i in range(3):
    sp.print_latex(sp.Eq(vector_system_moments_2_degenerate_spec_1[i], value_system_moments_2_degenerate_spec_1[i]))

In [ ]:
subs_to_moments_2_degenerate_spec_1 = dict(zip(vector_system_moments_2_degenerate_spec_1, value_system_moments_2_degenerate_spec_1))

In [ ]:
expr_m_j_eta_ik_degenerate_spec_1.subs(subs_to_moments_2_degenerate_spec_1).factor()

In [ ]:
expr_Q_star_degenerate_spec_1_from_moments_2 = expr_Q_star_degenerate_spec_1.subs(subs_to_moments_2_degenerate_spec_1).factor().expand()
expr_Q_star_degenerate_spec_1_from_moments_2

In [ ]:
expr = calc_degenerate_spec_1(expr_m_j_eta_ij_degenerate_spec_1)
expr.subs(subs_to_moments_2_degenerate_spec_1).factor().expand()

In [ ]:
expr = calc_degenerate_spec_1(Z3_degenerate_expr.diff(hi, lik)/(2*Z3_degenerate_expr))
expr.subs(subs_to_moments_2_degenerate_spec_1).factor().expand()

#### Проверка

In [ ]:
to_cF3 = lambda *args: cF3(*args[:4])
subs = {FFunction_spec_1: FFunction, EFunction_spec_1: EFunction}

f_expr_m_i_degenerate_spec_1 = sp.lambdify((h, l), expr_m_i_degenerate_spec_1.subs(subs), Z3sym.num_module_norm)
f_expr_m_j_degenerate_spec_1 = sp.lambdify((h, l), expr_m_j_degenerate_spec_1.subs(subs), Z3sym.num_module_norm)
f_expr_eta_ij_degenerate_spec_1 = sp.lambdify((h, l), expr_eta_ij_degenerate_spec_1.subs(subs), Z3sym.num_module_norm)
f_expr_eta_ik_degenerate_spec_1 = sp.lambdify((h, l), expr_eta_ik_degenerate_spec_1.subs(subs), Z3sym.num_module_norm)

args = 0.8, 1.1
compare_exprs(
    calc_numerical_diffs(calc_Z3_symmetrical_integrate, [args[0], args[0], args[1], args[1]])/(np.asarray([2, 1, 2, 1])*calc_Z3_symmetrical_integrate([args[0], args[0], args[1], args[1]])),
    calc_exprs([f_expr_m_i_degenerate_spec_1, f_expr_m_j_degenerate_spec_1, f_expr_eta_ij_degenerate_spec_1, f_expr_eta_ik_degenerate_spec_1], args)
)

In [ ]:
expr_m_i_degenerate_spec_1

In [ ]:
args = 1, 2.
moments = f_expr_m_i_degenerate_spec_1(*args), f_expr_eta_ij_degenerate_spec_1(*args),

for i in range(2):
    left = vector_system_moments_degenerate_spec_1[i].subs(subs)
    f_left = sp.lambdify((h, l, m, eta), left, Z3sym.num_module_norm)
    print(f_left(*args, *moments))

    right = vector_system_moments_degenerate_spec_1[i].subs(subs)
    f_right = sp.lambdify((h, l, m, eta), right, Z3sym.num_module_norm)
    print(f_right(*args, *moments))

    print()

In [ ]:
f_expr_Q_star_degenerate_spec_1 = sp.lambdify((h, l, m, eta), expr_Q_star_degenerate_spec_1.factor().subs(subs), Z3sym.num_module_norm)
f_expr_Q_star_degenerate_spec_1_from_moments_2 = sp.lambdify((h, l, m, eta, mjh2), expr_Q_star_degenerate_spec_1_from_moments_2.subs(subs), Z3sym.num_module_norm)
f_expr_m_par_2_degenerate_spec_1 = sp.lambdify((h, l, m, eta), expr_m_j_par_2_degenerate_spec_1.subs(subs), Z3sym.num_module_norm)

args = 49.03486660449675, 0.510352248405618
moments = f_expr_m_i_degenerate_spec_1(*args), f_expr_eta_ij_degenerate_spec_1(*args),

Q_star = nd.directionaldiff(calc_Z3_symmetrical_integrate, [args[0], args[0], args[1], args[1]], [0, 0, 1, 0], n=2)
Q_star /= 2*calc_Z3_symmetrical_integrate([args[0], args[0], args[1], args[1]])
Q_star -= f_expr_eta_ik_degenerate_spec_1(*args) + 1

print(Q_star)
print(f_expr_Q_star_degenerate_spec_1(*args, *moments))
print(f_expr_Q_star_degenerate_spec_1_from_moments_2(*args, *moments, f_expr_m_par_2_degenerate_spec_1(*args, *moments)))

## Генерация кода для метода Ньютона

### Определения

In [ ]:
class CPrinter(symbase.CPrinter):
    def _print_Function(self, expr):
        if isinstance(expr, Z2sym.F2):
            return f"dawson({self._print(expr.args[0])})"
        
        if isinstance(expr, F3):
            return f"F3({self._print(expr.args[0])}, {self._print(expr.args[1])}, {self._print(expr.args[2])})"

        return super()._print_Function(expr)
    

def get_ccode(expr):
    expr = symbase.optimize_pow(expr, {hi: "h_i", hj: "h_j", lij: "l_ij", lik: "l_ik"})
    return CPrinter().doprint(expr)

In [ ]:
def break_to_sub_exprs(name, expr, max_sub_exprs=60):
    args = expr.args
    N = len(args) // max_sub_exprs
    flag_first = True
    for _ in range(N):
        if flag_first:
            s = f"{name} = "
            flag_first = False
        else:
            s = f"{name} += "
        print(s, get_ccode(sp.Add(*args[:max_sub_exprs])), ";")
        args = args[max_sub_exprs:]

    if len(args) > 0:
        if flag_first:
            s = f"{name} = "
            flag_first = False
        else:
            s = f"{name} += "
        print(s, get_ccode(sp.Add(*args)), ";")

    print()

### Общий случай

In [ ]:
subs_exp = {
    sp.exp(-2*hi): sp.Symbol("_exp_2h_i"),
    sp.exp(-2*hj): sp.Symbol("_exp_2h_j"),
    sp.exp(-2*lij): sp.Symbol("_exp_2l_ij"),
    sp.exp(-2*lik): sp.Symbol("_exp_2l_ik")
}

subs_coeffs = {hi: sp.Symbol("h_i"), hj: sp.Symbol("h_j"), lij: sp.Symbol("l_ij"), lik: sp.Symbol("l_ik")}

subs_moments = {
    mi: sp.Symbol("mi"),
    mj: sp.Symbol("mj"),
    eta_ij: sp.Symbol("eta_ij"),
    eta_ik: sp.Symbol("eta_ik"),
    eta_ik_2: sp.Symbol("eta_ik2"),
    mj_eta_ij: sp.Symbol("mj_eta_ij"),
    mi_eta_ik: sp.Symbol("mi_eta_ik"),
    mj_eta_ik: sp.Symbol("mj_eta_ik")
}

subs = subs_exp
subs.update(subs_coeffs)
subs.update(subs_moments)

def to_norm(expr):
    expr *= sp.exp(-hj-lik-2*(hi+lij))
    return expr.expand()

In [ ]:
print("cF3 = ", get_ccode(to_norm(cF3(hi, hj, lij, lik).rewrite(sp.exp)).subs(subs)))

In [ ]:
def sigmas_args(arg_s1, arg_s2, arg_s3):
    result = ""
    if arg_s1:
        result += "1"
    if arg_s2:
        result += "2"
    if arg_s3:
        result += "3"
    return result


def sigmas_func_name(func):
    name = ""
    if isinstance(func, EFunction):
        name = "E"
    if isinstance(func, FFunction):
        name = "F"
    if isinstance(func, FTildeFunction):
        name = "FTilde"
    return name + sigmas_args(*func.args[-3:])

In [ ]:
subs_kappa = {
    lij*hi - lik*hj: 1/sp.Symbol("_kappa")
}

sigmas_funcs = []

def optimize_moment(expr):
    def add_sigmas_func(func):
        if func not in sigmas_funcs:
            sigmas_funcs.append(func)
        return sp.Symbol(sigmas_func_name(func))*cF3(hi, hj, lij, lik)*sp.Symbol("_cF3")
    expr = expr.subs(subs_kappa).expand()
    expr = expr.replace(EFunction, lambda *args: add_sigmas_func(EFunction(*args)))
    expr = expr.replace(FFunction, lambda *args: add_sigmas_func(FFunction(*args)))
    expr = expr.replace(FTildeFunction, lambda *args: add_sigmas_func(FTildeFunction(*args)))
    return expr

In [ ]:
break_to_sub_exprs("mi", optimize_moment(expr_mi).subs(subs), 7)
break_to_sub_exprs("mj", optimize_moment(expr_mj).subs(subs), 7)
break_to_sub_exprs("eta_ij", optimize_moment(expr_eta_ij).subs(subs), 7)
break_to_sub_exprs("eta_ik", optimize_moment(expr_eta_ik).subs(subs), 7)

expr = expr_mj_eta_ij.subs(subs_det_system_moments).factor().expand()
expr = sp.Add(*map(sp.factor, expr.args))
break_to_sub_exprs("mj_eta_ij", optimize_moment(expr).subs(subs), 5)

expr = expr_mi_eta_ik.subs(subs_det_system_moments).factor().expand()
expr = sp.Add(*map(sp.factor, expr.args))
break_to_sub_exprs("mi_eta_ik", optimize_moment(expr).subs(subs), 5)

expr = expr_mj_eta_ik.subs(subs_det_system_moments).factor().expand()
expr = sp.Add(*map(sp.factor, expr.args))
break_to_sub_exprs("mj_eta_ik", optimize_moment(expr).subs(subs), 5)

expr = expr_eta_ik_2.subs(subs_det_system_moments).factor().expand()
expr = sp.Add(*map(sp.factor, expr.args))
break_to_sub_exprs("eta_ik2", optimize_moment(expr).subs(subs), 5)

In [ ]:
for sigmas_func in sigmas_funcs:
    print(sigmas_func_name(sigmas_func), end=", ")

In [ ]:
for sigmas_func in sigmas_funcs:
    print(f"{sigmas_func_name(sigmas_func)} = {get_ccode(to_norm(sigmas_func.rewrite(sp.exp)).subs(subs_exp))}")

In [ ]:
print(expr_Q_star_from_m_j_eta_ij.subs(subs))

### Вырожденный случай

In [ ]:
# sigmas_funcs = []

def optimize_moment_degenerate(expr):
    def add_sigmas_func(func):
        if func not in sigmas_funcs:
            sigmas_funcs.append(func)
        return sp.Symbol(sigmas_func_name(func))*cF3(hi, hj, lij, lik)*sp.Symbol("_cF3")
    
    expr = evaluate_cF3_degenerate(expr).subs(subs_to_cF3Function_degenerate).expand()
    expr = expr.replace(cF3_degenerate, lambda *args: cF3(*args[:4]))
    expr = expr.replace(EFunction, lambda *args: add_sigmas_func(EFunction(*args)))
    expr = expr.replace(FFunction, lambda *args: add_sigmas_func(FFunction(*args)))
    return expr

In [ ]:
# def _calc_degenerate(expr):
#     expr = expr.expand()
#     expr = evaluate_cF3_degenerate(expr)
#     expr = expr.subs(subs_to_cF3Function_degenerate).expand()
#     return expr

# _expr_m_j_eta_ij_degenerate = _calc_degenerate(Z3_degenerate_expr.diff(hj, lij)/(2*Z3_degenerate_expr))
# _expr_m_i_eta_ik_degenerate = _calc_degenerate(Z3_degenerate_expr.diff(hi, lik)/(2*Z3_degenerate_expr))
# _expr_m_j_eta_ik_degenerate = _calc_degenerate(Z3_degenerate_expr.diff(hj, lik)/(Z3_degenerate_expr))
# _expr_eta_ik_2_degenerate = _calc_degenerate(Z3_degenerate_expr.diff(lik, 2)/(Z3_degenerate_expr))

In [ ]:
# break_to_sub_exprs("mi", optimize_moment_degenerate(expr_m_i_degenerate).subs(subs), 7)
# break_to_sub_exprs("mj", optimize_moment_degenerate(expr_m_j_degenerate).subs(subs), 7)
# break_to_sub_exprs("eta_ij", optimize_moment_degenerate(expr_eta_ij_degenerate).subs(subs), 7)
# break_to_sub_exprs("eta_ik", optimize_moment_degenerate(expr_eta_ik_degenerate).subs(subs), 7)

# break_to_sub_exprs("mj_eta_ij", optimize_moment_degenerate(_expr_m_j_eta_ij_degenerate).subs(subs), 7)
# break_to_sub_exprs("mi_eta_ik", optimize_moment_degenerate(_expr_m_i_eta_ik_degenerate).subs(subs), 7)
# break_to_sub_exprs("mj_eta_ik", optimize_moment_degenerate(_expr_m_j_eta_ik_degenerate).subs(subs), 7)
# break_to_sub_exprs("eta_ik2", optimize_moment_degenerate(_expr_eta_ik_2_degenerate).subs(subs), 7)

In [ ]:
for sigmas_func in sigmas_funcs:
    print(sigmas_func_name(sigmas_func), end=", ")

In [ ]:
for sigmas_func in sigmas_funcs:
    print(f"{sigmas_func_name(sigmas_func)} = {get_ccode(to_norm(sigmas_func.rewrite(sp.exp)).subs(subs))}")

### Полностью симметричный случай

In [ ]:
# sigmas_funcs = []

def optimize_moment_degenerate_spec_1(expr):
    def add_sigmas_func(func):
        if func not in sigmas_funcs:
            sigmas_funcs.append(func)
        return sp.Symbol(sigmas_func_name(func))#*cF3(hi, hj, lij, lik)*sp.Symbol("_cF3")
    
    expr = evaluate_cF3_degenerate(expr).expand()
    # expr = expr.subs(subs_to_cF3Function_degenerate_spec_1).expand()
    expr = expr.replace(cF3_degenerate, lambda *args: cF3(hi, hj, lij, lik))
    expr = expr.replace(EFunction_spec_1, lambda *args: add_sigmas_func(EFunction(*args)))
    expr = expr.replace(FFunction_spec_1, lambda *args: add_sigmas_func(FFunction(*args)))
    return expr.subs({h: hi, l: lij})

In [ ]:
# def _calc_degenerate_spec_1(expr):
#     expr = expr.expand()
#     expr = evaluate_cF3_degenerate(expr)
#     expr = expr.replace(EFunction, EFunction_spec_1).replace(FFunction, FFunction_spec_1)
#     expr = expr.subs({hi: h, hj: h, lij: l, lik: l})
#     return expr


# _expr_m_j_eta_ij_degenerate_spec_1 = _calc_degenerate_spec_1(Z3_degenerate_expr.diff(hj, lij)/(2*Z3_degenerate_expr))
# _expr_eta_ik_2_degenerate_spec_1 = _calc_degenerate_spec_1(Z3_degenerate_expr.diff(lik, 2)/(Z3_degenerate_expr))

In [ ]:
# break_to_sub_exprs("mi", optimize_moment_degenerate_spec_1(expr_m_i_degenerate_spec_1).subs(subs), 7)
# break_to_sub_exprs("eta_ij", optimize_moment_degenerate_spec_1(expr_eta_ij_degenerate_spec_1).subs(subs), 7)

In [ ]:
# break_to_sub_exprs("mj_eta_ij", optimize_moment_degenerate_spec_1(_expr_m_j_eta_ij_degenerate_spec_1).subs(subs), 7)
# break_to_sub_exprs("eta_ik2", optimize_moment_degenerate_spec_1(_expr_eta_ik_2_degenerate_spec_1).subs(subs), 7)

## Решение обратной задачи

In [ ]:
%timeit Z3num.calc_from_coeffs_symmetrical(.1, .1, .1, .1)

In [ ]:
%timeit Z3num.calc_Z3_integrate(1, 1, 1, 1, 1, 1)

### Обратная задача в случае $\left<m_i\right>=\left<m_j\right>$

In [ ]:
N = 32
left, right = 0, 1

m = np.linspace(left, right, N)
zeta_1 = np.linspace(left, right, N)
zeta_2 = np.linspace(left, right, N)

m, zeta_1, zeta_2 = np.meshgrid(m, zeta_1, zeta_2)

zeta_ij = zeta_1
zeta_ik = zeta_2 + (1-zeta_2)*zeta_ij**2

eta_ij = zeta_ij + (1-zeta_ij)*m**2
eta_ik = zeta_ik + (1-zeta_ik)*m**2

In [ ]:
hi, hj, lij, lik = np.empty_like(m), np.empty_like(m), np.empty_like(m), np.empty_like(m)
mj_eta_ij, mi_eta_ik, mj_eta_ik, eta_ik2 = np.empty_like(m), np.empty_like(m), np.empty_like(m), np.empty_like(m)
mjh2, Q_star = np.empty_like(m), np.empty_like(m)
Z3_norm = np.empty_like(m)

for i in range(m.size):
    z3 = Z3num.find_coeffs_symmetrical(m.flat[i], eta_ij.flat[i], eta_ik.flat[i])
    hi.flat[i], hj.flat[i], lij.flat[i], lik.flat[i]  = z3.h_i, z3.h_j, z3.l_ij, z3.l_ik
    mj_eta_ij.flat[i], mi_eta_ik.flat[i], mj_eta_ik.flat[i], eta_ik2.flat[i] = z3.mj_eta_ij, z3.mi_eta_ik, z3.mj_eta_ik, z3.eta_ik2
    mjh2.flat[i], Q_star.flat[i] = z3.mjh2, z3.Q_star
    Z3_norm.flat[i] = z3.Z3_norm

In [ ]:
def write_frame(file, name, array):  
    file.write(f"SCALARS {name} float 1\n")
    file.write("LOOKUP_TABLE default\n")
    for k in range(array.shape[2]):
        for j in range(array.shape[1]):
            for i in range(array.shape[0]):
                value = array[j, i, k]
                file.write(f"{value}\n")

file = open("data/Z3.vtk", "w")
file.write("# vtk DataFile Version 2.0\n")
file.write("Z3\n")
file.write("ASCII\n")
file.write("DATASET STRUCTURED_POINTS\n")
file.write(f"DIMENSIONS {N} {N} {N}\n")
file.write(f"SPACING {(right-left)/(N-1)} {(right-left)/(N-1)} {(right-left)/(N-1)}\n")
file.write(f"ORIGIN {left} {left} {left}\n")
file.write(f"POINT_DATA {N**3}\n")

write_frame(file, "m", m)
write_frame(file, "zeta_1", zeta_1)
write_frame(file, "zeta_2", zeta_2)

write_frame(file, "hi", hi)
write_frame(file, "hj", hj)
write_frame(file, "lij", lij)
write_frame(file, "lik", lik)

write_frame(file, "mj_eta_ij", mj_eta_ij)
write_frame(file, "mi_eta_ik", mi_eta_ik)
write_frame(file, "mj_eta_ik", mj_eta_ik)
write_frame(file, "eta_ik2", eta_ik2)

write_frame(file, "mjh2", mjh2)
write_frame(file, "Q_star", Q_star)

write_frame(file, "Z3_norm", Z3_norm)

file.close()

In [ ]:
%%bash
vtk2msh data/Z3.vtk

### Полностью симметричный случай

In [ ]:
z3 = Z3num.find_coeffs_symmetrical_by_scipy(.6, .4, .4)
z3.h_i, z3.h_j, z3.l_ij, z3.l_ik

In [ ]:
z3 = Z3num.find_coeffs_symmetrical(.6, .4, .4)
z3.h_i, z3.h_j, z3.l_ij, z3.l_ik

In [ ]:
m = np.linspace(0, 1, 100)
zeta = np.linspace(0, 1, 100)

m, zeta = np.meshgrid(m, zeta)
eta = zeta + (1-zeta)*m**2

In [ ]:
p, h, l = np.empty_like(m), np.empty_like(m), np.empty_like(m)
upsilon, eta2, mhj = np.empty_like(m), np.empty_like(m), np.empty_like(m)
psi0, Z2_norm = np.empty_like(m), np.empty_like(m)

for i in range(m.size):
    z2 = Z2num.find_coeffs_symmetrical(m.flat[i], eta.flat[i])
    h.flat[i], l.flat[i] = z2.h, z2.l
    upsilon.flat[i], eta2.flat[i], mhj.flat[i] = z2.upsilon, z2.eta2, z2.mh2
    psi0.flat[i], Z2_norm.flat[i] = z2.psi0, z2.Z2_norm
    p.flat[i] = Z2num.invL(m.flat[i])

In [ ]:
hi, hj, lij, lik = np.empty_like(m), np.empty_like(m), np.empty_like(m), np.empty_like(m)
mj_eta_ij, mi_eta_ik, mj_eta_ik, eta_ik2 = np.empty_like(m), np.empty_like(m), np.empty_like(m), np.empty_like(m)
mjh2, Q_star = np.empty_like(m), np.empty_like(m)
Z3_norm = np.empty_like(m)

for i in range(m.size):
    z3 = Z3num.find_coeffs_symmetrical(m.flat[i], eta.flat[i], eta.flat[i])
    hi.flat[i], hj.flat[i], lij.flat[i], lik.flat[i]  = z3.h_i, z3.h_j, z3.l_ij, z3.l_ik
    mj_eta_ij.flat[i], mi_eta_ik.flat[i], mj_eta_ik.flat[i], eta_ik2.flat[i] = z3.mj_eta_ij, z3.mi_eta_ik, z3.mj_eta_ik, z3.eta_ik2
    mjh2.flat[i], Q_star.flat[i] = z3.mjh2, z3.Q_star
    Z3_norm.flat[i] = z3.Z3_norm

In [ ]:
datlib.np2dat(
    "data/Z3_all_symmetrical.dat",
    "m eta zeta Q".split(),
    np.asarray([
        m, eta, zeta, Q_star
    ])
)

### Асимптотика при $\left<m\right> \rightarrow 1$ 

In [ ]:
N = 32

m = np.linspace(.99, 1, N)
eta_ij = np.linspace(.99, 1, N)
eta_ik = np.linspace(.99, 1, N)

m, eta_ij, eta_ik = np.meshgrid(m, eta_ij, eta_ik)

In [ ]:
hi, hj, lij, lik = np.empty_like(m), np.empty_like(m), np.empty_like(m), np.empty_like(m)
mj_eta_ij, mi_eta_ik, mj_eta_ik, eta_ik2 = np.empty_like(m), np.empty_like(m), np.empty_like(m), np.empty_like(m)
mjh2, Q_star = np.empty_like(m), np.empty_like(m)
Z3_norm = np.empty_like(m)

for i in range(m.size):
    z3 = Z3num.find_coeffs_symmetrical(m.flat[i], eta_ij.flat[i], eta_ik.flat[i])
    hi.flat[i], hj.flat[i], lij.flat[i], lik.flat[i]  = z3.h_i, z3.h_j, z3.l_ij, z3.l_ik
    mj_eta_ij.flat[i], mi_eta_ik.flat[i], mj_eta_ik.flat[i], eta_ik2.flat[i] = z3.mj_eta_ij, z3.mi_eta_ik, z3.mj_eta_ik, z3.eta_ik2
    mjh2.flat[i], Q_star.flat[i] = z3.mjh2, z3.Q_star
    Z3_norm.flat[i] = z3.Z3_norm

In [ ]:
def write_frame(file, name, array):  
    file.write(f"SCALARS {name} float 1\n")
    file.write("LOOKUP_TABLE default\n")
    for k in range(array.shape[2]):
        for j in range(array.shape[1]):
            for i in range(array.shape[0]):
                value = array[j, i, k]
                file.write(f"{value}\n")

file = open("data/Z3.vtk", "w")
file.write("# vtk DataFile Version 2.0\n")
file.write("Z3\n")
file.write("ASCII\n")
file.write("DATASET STRUCTURED_POINTS\n")
file.write(f"DIMENSIONS {N} {N} {N}\n")
file.write(f"SPACING {.01/(N-1)} {.01/(N-1)} {.01/(N-1)}\n")
file.write(f"ORIGIN .99 .99 .99\n")
file.write(f"POINT_DATA {N**3}\n")

write_frame(file, "hi", hi)
write_frame(file, "h_j", hj)
write_frame(file, "lij", lij)
write_frame(file, "lik", lik)

write_frame(file, "Q_star", Q_star)

file.close()

In [ ]:
%%bash
vtk2msh data/Z3.vtk

In [ ]:
variables = 1-m, 1-eta_ij, 1-eta_ik
y_data = mjh2
indexes_nans = np.logical_not(np.isnan(hi))

def func(_, *p_args, nans=True):    
    result = p_args[0]
    result += p_args[1]*variables[0] + p_args[2]*variables[1] + p_args[3]*variables[2]

    if nans:
        return result[indexes_nans].ravel()
    else:
        return result
    

p0 = [0] + 3*[1]
p = opt.curve_fit(func, None, y_data[indexes_nans].ravel(), p0=p0)[0]
p

In [ ]:
# x = 1 - m
# y = 1 - eta_ij
# z = 1 - eta_ik

# Q_star = -4y + z => Q = -2y + z
# mj_eta_ij = 1 - x - y
# mi_eta_ik = 1 - x - z
# mj_eta_ik = 1 - x - z
# eta_ik2 = 1 - 2z
# mjh2 = 1 - 2x